In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats)
import time

In [2]:
pbp2023=pd.read_csv(filepath_or_buffer="Data/2022-23_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [3]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2022-23",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season="2022-23",
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [4]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [5]:
hustle_70percent_gp.nlargest(10,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
285,Kevin Love,0.967164
276,Kenrich Williams,0.796313
347,Montrezl Harrell,0.789474
72,Clint Capela,0.576000
382,Patrick Beverley,0.508550
94,Davion Mitchell,0.498462
65,Chris Boucher,0.459912
168,Isaiah Joe,0.453782
163,Herbert Jones,0.443662
451,Trendon Watford,0.434483


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [6]:
hustle_70percent_gp.nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
222,Jericho Sims,1.785600
478,Wenyen Gabriel,1.675000
428,T.J. McConnell,1.593103
432,Tari Eason,1.534737
73,Coby White,1.310680
386,Paul Reed,1.293204
168,Isaiah Joe,1.270588
276,Kenrich Williams,1.260829
385,Paul George,1.235191
368,O.G. Anunoby,1.213043


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(10,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
11,Alex Caruso,5.515200
211,Javonte Green,5.187578
249,Josh Okogie,4.929730
428,T.J. McConnell,4.903448
100,De'Anthony Melton,4.548000
439,Thaddeus Young,4.464000
385,Paul George,4.222874
432,Tari Eason,4.206316
160,Hamidou Diallo,4.200000
386,Paul Reed,4.159223


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
426,Steven Adams,16.617490
31,Bismack Biyombo,16.471942
288,Kevon Looney,14.887336
261,Jusuf Nurkic,14.626573
371,Onyeka Okongwu,14.235616
312,Luke Kornet,13.978125
72,Clint Capela,13.666909
193,Jalen Duren,13.637288
405,Rudy Gobert,13.588571
328,Mason Plumlee,13.562182


# The No Fly Zone Award (presented by Dikembe Mutumbo)*

most blocked dunks as the blocking player

In [9]:
misses=pbp2023[pbp2023.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp2023.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
#as the blocker
blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
12,Brook Lopez,10
50,Jarrett Allen,7
87,Myles Turner,7
21,Daniel Gafford,6
66,Kenyon Martin Jr.,6
23,Darius Bazley,5
90,Nick Richards,5
17,Christian Koloko,4
42,Jabari Smith Jr.,4
47,Jalen Smith,4


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop\_soup for the idea and asw7412 for the sponsor)

In [10]:
#as the blockee
blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
0,Aaron Gordon,6
48,Ivica Zubac,6
128,Zion Williamson,6
21,Chris Boucher,5
98,Nick Richards,5
114,Tari Eason,5
4,Anthony Davis,4
59,Jarrett Allen,4
66,Joel Embiid,4
116,Terance Mann,4


# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears, presented by Gary Payton & Shawn Kemp)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [11]:
#get alley oops
alley_oops=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('Alley',na=False)]
#remove missed alley oop attempts
made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
#remove alley oops that have missing 2nd player (self-alley oop or perhaps missing data?)
made_alley_oops_complete=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
made_alley_oops_complete['p1']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
made_alley_oops_complete['p2']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)
made_alley_oops_complete.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,p1,p2,count
120,Clint Capela,Trae Young,35
317,John Collins,Trae Young,14
29,Anthony Davis,Russell Westbrook,12
68,Brandon Clarke,Ja Morant,12
146,Darius Garland,Evan Mobley,11
200,Dwight Powell,Luka Doncic,11
14,Andre Drummond,Goran Dragic,8
44,Bam Adebayo,Tyler Herro,8
212,Franz Wagner,Wendell Carter Jr.,8
269,Jalen Brunson,Mitchell Robinson,8


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [12]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS_2PT','CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)

In [13]:
hustle_50percent_gp_ranks.to_csv(path_or_buf="Output Data/Hustle Ranks.csv")

In [14]:
hustle_50percent_gp_ranks.nsmallest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
135,Eric Gordon,0.271552,0.008621,0.051724,0.293103,0.057471,0.264368,0.946839
293,Klay Thompson,0.271552,0.195402,0.048851,0.359195,0.048851,0.241379,1.165230
77,Collin Sexton,0.637931,0.155172,0.170977,0.106322,0.028736,0.206897,1.306034
236,Jordan Clarkson,0.271552,0.327586,0.192529,0.327586,0.068966,0.189655,1.377874
86,Damion Lee,0.271552,0.209770,0.224138,0.126437,0.330460,0.321839,1.484195
466,Tyus Jones,0.821839,0.002874,0.270115,0.275862,0.112069,0.063218,1.545977
126,Doug McDermott,0.271552,0.341954,0.238506,0.005747,0.390805,0.298851,1.547414
394,Reggie Bullock,0.271552,0.097701,0.025862,0.356322,0.272989,0.543103,1.567529
339,Michael Porter Jr.,0.271552,0.373563,0.209770,0.456897,0.045977,0.215517,1.573276
144,Furkan Korkmaz,0.271552,0.017241,0.252874,0.201149,0.333333,0.560345,1.636494


In [15]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME'].isin(['Patrick Beverley','Russell Westbrook'])].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
382,Patrick Beverley,0.985632,0.106322,0.060345,0.781609,0.074713,0.129310,2.137931
407,Russell Westbrook,0.804598,0.198276,0.416667,0.675287,0.267241,0.244253,2.606322


# The "Got that Dawg in Him" Award (presented by Air Bud)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [16]:
hustle_50percent_gp_ranks.nlargest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
163,Herbert Jones,0.974138,0.652299,0.862069,0.962644,0.683908,0.905172,5.040230
24,Anthony Lamb,0.942529,0.781609,0.818966,0.577586,0.801724,0.910920,4.833333
386,Paul Reed,0.271552,0.965517,0.853448,0.965517,0.836207,0.939655,4.831897
171,Isaiah Roby,0.902299,0.712644,0.864943,0.660920,0.652299,0.948276,4.741379
167,Isaiah Jackson,0.718391,0.913793,0.959770,0.775862,0.804598,0.505747,4.678161
439,Thaddeus Young,0.893678,0.623563,0.790230,0.979885,0.681034,0.689655,4.658046
11,Alex Caruso,0.936782,0.479885,0.954023,0.997126,0.298851,0.985632,4.652299
21,Anthony Davis,0.609195,0.862069,0.622126,0.879310,0.772989,0.847701,4.593391
469,Usman Garuba,0.683908,0.839080,0.997126,0.724138,0.643678,0.705460,4.593391
127,Draymond Green,0.844828,0.816092,0.948276,0.850575,0.922414,0.186782,4.568966


# The Bowling Ball Award (sponsored by [Pete Weber](https://www.youtube.com/watch?v=gKQOXYB2cd8), presented by Glen "Big Baby" Davis)*

most charges committed (credit to Kdog122025 for the idea)

In [17]:
charges=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Charge',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('Charge',na=False)]

charges.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
7,Anthony Edwards,9
59,Giannis Antetokounmpo,8
147,O.G. Anunoby,8
11,Bam Adebayo,7
95,Joel Embiid,7
124,Kyle Kuzma,7
192,Zion Williamson,7
12,Bennedict Mathurin,6
71,Jalen Brunson,6
151,Pascal Siakam,6


# "The Good Ol' Hockey Game, is the Best Game You Can Name" Award (presented by Dominik Hasek)*

most goaltends committed (credit to Kdog122025 for the idea)

In [18]:
goaltends=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Goaltending',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('Goaltending',na=False)]
goaltends.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
49,Jalen Duren,9
66,John Collins,6
104,Nick Richards,6
54,Jaren Jackson Jr.,5
87,LeBron James,5
0,Aaron Gordon,4
9,Bam Adebayo,4
18,Charles Bassey,4
22,Clint Capela,4
44,Isaiah Stewart,4


# The Most 3-Pt Shooting Fouls Committed Award*

credit to watchingsongsDL, kingcobweb & An-Indian-In-The-NBA for the idea

In [19]:
third_free_throws=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)|
                          pbp2023['VISITORDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)]

shooting_fouls=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('S.FOUL',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('S.FOUL',na=False)]

fouls_on_missed_threes=shooting_fouls.merge(right=third_free_throws,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

three_pointers=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('3PT',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('3PT',na=False)]

made_threes=three_pointers[~three_pointers.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~three_pointers.VISITORDESCRIPTION.str.startswith('MISS',na=False)]

fouls_on_made_threes=shooting_fouls.merge(right=made_threes,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

fouls_on_threes=fouls_on_missed_threes.append(fouls_on_made_threes)
fouls_on_threes.groupby('PLAYER1_NAME_x').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME_x,count
158,Patrick Beverley,5
4,Alex Caruso,4
34,Coby White,4
47,De'Andre Hunter,4
48,De'Anthony Melton,4
52,Deni Avdija,4
57,Dillon Brooks,4
89,Jalen McDaniels,4
119,Kevin Durant,4
125,Kyle Kuzma,4


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award

Lowest contested rebound percentage, minimum 50% of games played

In [20]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
31,Patty Mills,0.000
410,Duane Washington Jr.,0.000
115,Tyus Jones,0.056
87,Seth Curry,0.067
336,Malachi Flynn,0.067
248,Kendrick Nunn,0.069
164,Danuel House Jr.,0.074
60,Austin Rivers,0.079
114,T.J. McConnell,0.088
14,Eric Gordon,0.093


alternatively: restricting to players > 6 foot 6 inches in height

In [22]:
player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
156,Terrence Ross,0.111
140,Paul George,0.168
84,Jordan Nwora,0.173
66,Jalen McDaniels,0.180
74,Jerami Grant,0.189
13,Bojan Bogdanovic,0.198
98,Kevin Huerter,0.198
72,Jayson Tatum,0.209
148,Saddiq Bey,0.213
4,Amir Coffey,0.219


# The "Where There's a Will, There's a Way" Award (presented by Dennis Rodman)

Highest contested rebound percentage, minimum 50% of games played

In [23]:
reb_w_gp_qualify.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
246,Wenyen Gabriel,0.659
16,Robin Lopez,0.656
226,Mitchell Robinson,0.638
274,Romeo Langford,0.588
194,Luke Kornet,0.584
281,Moses Brown,0.583
376,Isaiah Jackson,0.573
54,Anthony Davis,0.552
379,Day'Ron Sharpe,0.552
447,Walker Kessler,0.544


alternatively: restricting to players < 6 foot 7 inches in height

In [24]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
147,Romeo Langford,0.588
97,Josh Okogie,0.494
46,Derrick Jones Jr.,0.458
83,Javonte Green,0.455
105,Kenrich Williams,0.442
174,Zion Williamson,0.430
108,Kevon Harris,0.408
95,Josh Christopher,0.400
100,Justin Holiday,0.400
3,Aaron Wiggins,0.396


# The "David vs Goliath" Award (presented by Dwyane Wade)*

most shots blocked with at least a 5-inch difference in height

In [25]:
blocks=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('BLOCK',na=False)|
                          pbp2023['VISITORDESCRIPTION'].str.contains('BLOCK',na=False)]

blocking_player_heights=blocks.merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER1_ID',right_on='PLAYER_ID').merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER3_ID',right_on='PLAYER_ID')
blocking_player_heights["height_difference"]=blocking_player_heights['PLAYER_HEIGHT_INCHES_x']-blocking_player_heights['PLAYER_HEIGHT_INCHES_y']
blocking_player_heights.query('height_difference >= 5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
102,Kyle Lowry,10
52,Fred VanVleet,9
45,Donovan Mitchell,8
103,Kyrie Irving,8
25,D'Angelo Russell,5
73,Javonte Green,5
77,Jevon Carter,5
83,Jordan Goodwin,5
99,Kevin Porter Jr.,5
121,Patrick Beverley,5


# The Rotation Awards

**value depth over one solitary star**

# The Best Guard Rotation Award (sponsored by Buckingham Palace)

# The Best Wing Rotation Award (co-sponsored by Lou Williams and Magic City)

# The Best Big Rotation Award (jointly sponsored by Tom Hanks, Cadbury and Sex and the City)